In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
from sklearn import datasets, neighbors, metrics, tree, svm, preprocessing, model_selection, ensemble
from sklearn.base import clone as sklearn_clone
from sklearn.model_selection import StratifiedKFold
from pprint import pprint

In [2]:
%%time
df = pd.read_pickle("../data/Lauren/covid_df.pkl")
labels = pd.read_pickle("../data/Lauren/covid_labels.pkl")
df.head()

CPU times: user 16.4 ms, sys: 1.88 s, total: 1.9 s
Wall time: 1.93 s


,AL627309.1,AL627309.4,AL669831.5,FAM87B,LINC00115,FAM41C,AL645608.3,SAMD11,NOC2L,KLHL17,...,AC005828.2,LINC01977,MEP1B,CSTL1,AL034550.1,JPH2,PRODH2,PSG5,TEX101,AC007663.2
COV013_AAACCCAAGCATCCCG-1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
COV013_AAACCCAAGTGGATTA-1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
COV013_AAACCCACACAACATC-1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
COV013_AAACCCACACGCCAGT-1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
COV013_AAACCCACAGACGATG-1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
# df.describe()

## 1. Removing all-zero columns

In [4]:
nonzero = df.sum() > 0
print(nonzero)
print(sum(nonzero))
df = df.loc[:,nonzero]
df.shape

AL627309.1     True
AL627309.4    False
AL669831.5     True
FAM87B         True
LINC00115      True
              ...  
JPH2          False
PRODH2        False
PSG5          False
TEX101        False
AC007663.2    False
Length: 24444, dtype: bool
21044


(19240, 21044)

In [5]:
df.to_pickle(  "../data/Lauren/covid_df_filtered.pkl")

## 2. Low counts

There are columns that have mostly zeros, except for a few other numbers. Filter those columns out.

In [6]:

def remove_less_than_X_nonzeros(df, X):
    a = []
    for (columnName, columnData) in df.iteritems():
        counts = df[columnName].value_counts() # This is sorted with the highest first (a lot of zeros)
        sum_of_others = df.shape[0] - counts[0]
        if sum_of_others < X:
            a.append(columnName)
    print(f"Removing {len(a)} columns")
    return df.drop(columns=a)


In [7]:
df_10   = remove_less_than_X_nonzeros(df, 10)
df_20   = remove_less_than_X_nonzeros(df, 20)
df_100  = remove_less_than_X_nonzeros(df, 100)
df_1000 = remove_less_than_X_nonzeros(df, 1000)

Removing 2953 columns
Removing 4519 columns
Removing 7809 columns
Removing 13185 columns


In [8]:
df_10.to_pickle(  "../data/Lauren/covid_df_filtered_X_0010.pkl")
del df_10
df_20.to_pickle(  "../data/Lauren/covid_df_filtered_X_0020.pkl")
del df_20
df_100.to_pickle( "../data/Lauren/covid_df_filtered_X_0100.pkl")
del df_100
df_1000.to_pickle("../data/Lauren/covid_df_filtered_X_1000.pkl")